In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sage.all_cmdline import *

from ctmc.metrics import *
from ctmc.generator import *

In [2]:
from ctmc.model import *

In [3]:
par = {k: 100, gamm: 0.00083, beta: 0.314, omegaA: 60, omegaB: 40}
rp = {pA: 0.5, pB: 0.5, pAB: 0.33, pBA: 0.33}

In [14]:
fake_pi = vector(var("π1 π2 π3 π4 π5 π6 π7 π8 π9"))

In [4]:
pi.subs(par)

(0.45730066177094375, 0.1990406321317778, 0.1444722431826512, 0.09409979741795944, 0.02812322552195045, 0.05712438786191808, 0.014593817040464939, 0.003849743350723288, 0.0013954917216112573)

In [12]:
(mining_throughput(pi) == total_throughput(pi)).subs(par)

0.09326036765671758 == 0.09326036765671758

In [21]:
(mining_throughput(fake_pi) == total_throughput(fake_pi)).simplify()

k*γ*(π1 + π3 + π4 + π6) + 2*γ*(π3 + π6 + π8)*ωA + 2*γ*(π4 + π6 + π9)*ωB + γ*π2 == -((pA - 1)*π3 + (pB - 1)*π4 + (pAB + pBA - 1)*π6)*k*γ + (pA*π3 + pAB*π6)*k*γ + (pB*π4 + pBA*π6)*k*γ + k*γ*π1 + 2*γ*(π3 + π6 + π8)*ωA + 2*γ*(π4 + π6 + π9)*ωB + γ*π2

In [17]:
print(poolA_throughput(fake_pi))
print(poolB_throughput(fake_pi))
print(fair_throughput(fake_pi))

(pA*π3 + pAB*π6)*k*γ + 2*γ*(π3 + π6 + π8)*ωA
(pB*π4 + pBA*π6)*k*γ + 2*γ*(π4 + π6 + π9)*ωB
-((pA - 1)*π3 + (pB - 1)*π4 + (pAB + pBA - 1)*π6)*k*γ + k*γ*π1 + γ*π2


In [13]:
pi.subs(par | {omegaA: 100, omegaB: 0})

(0.47596397318210937, 0.1946716813379216, 0.26050479718062997, 1.1547053248068778e-16, 0.05695258473924456, 5.773526624034389e-17, 0, 0.011906963560093952, 6.104493118405789e-17)

In [ ]:
v = var('v')
tot = 200
f = mining_throughput(pi).subs(par | rp | {omegaA: v * tot, omegaB: v / 1.5 * tot, k: (1 - v * 1.66) * tot}).simplify()

In [ ]:
x_values = np.linspace(0, 0.3, 10)
y_values = np.array([f.subs({v: x}) for x in x_values])

fig, ax = plt.subplots()

ax.plot(x_values, y_values, label='Throughput')

pass